In [ ]:
%load_ext autoreload
%autoreload 2

#%config IPCompleter.greedy=True

# Rotation curves

First, imports:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
from astropy import units
from pathlib import Path
import os
import astropy.units as u
from astropy.constants import G

import snapshot_obj 
import subhalo
import dataset_compute
import curve_fit

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(subhalo)
importlib.reload(dataset_compute)
importlib.reload(curve_fit)

## Get data

In [ ]:
snap_id = 127
sim_ids = ["V1_LR_fix", "V1_LR_curvaton_p082_fix", "V1_LR_curvaton_p084_fix"]
names = ["LCDM", "p082", "p084"]

# Which halo:
halo = [(1,0), (1,0), (1,0)]

data = {}
for name, sim_id, halo_id in zip(names, sim_ids, halo):
    data[name] = {"snapshot": snapshot_obj.Snapshot(sim_id, snap_id, name=name),
                  "halo_identifier": halo_id}

In [ ]:
for name, sim_data in data.items():
    snap = sim_data["snapshot"]
    idx = snap.index_of_halo(sim_data["halo_identifier"][0], 
                             sim_data["halo_identifier"][1])

    rot_curves = snap.get_subhalos('Vcirc', 
                                   group='Extended/RotationCurve/All')
    sub_offset = snap.get_subhalos('SubOffset', 
                                   group='Extended/RotationCurve/All')
    v_circ = rot_curves[:,0] * units.cm.to(units.km)
    radii = rot_curves[:,1] * units.cm.to(units.kpc)
    
    max_point = snap.get_subhalos("Max_Vcirc", group="Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
    rmax = max_point[:,1] * units.cm.to(units.kpc)
    
    data[name]["Vmax"] = vmax[idx]
    data[name]["Rmax"] = rmax[idx]
    data[name]["Vcirc"] = v_circ[sub_offset[idx]:sub_offset[idx+1]]
    data[name]["Radius"] = radii[sub_offset[idx]:sub_offset[idx+1]]

In [ ]:
# Set some parameters:
x_down = 0; x_up = 200
y_down = 0; y_up = 300

color = ["black", "red", "blue", "green"]

In [ ]:
# Construct saving location:
filename = 'rotation_curve'
for name, sim_data in data.items():
    filename += "_{}_gn{}sgn{}".format(name, sim_data["halo_identifier"][0],
                                       sim_data["halo_identifier"][1])
filename += ".png"
    
home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures")
filename = os.path.join(path, filename)

In [ ]:
fig, axes = plt.subplots()

# Set axes:
axes.set_xlim(x_down, x_up)
axes.set_ylim(y_down,y_up)

# Set labels:
axes.set_xlabel('$r[\mathrm{kpc}]$', fontsize=16)
axes.set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$', fontsize=16)

# Galaxy text:
axes.set_title('Rotation curve')

for i, (name, entry) in enumerate(data.items()):
    lab = "{}: ({}, {})".format(name, 
                                entry["halo_identifier"][0],
                                entry["halo_identifier"][1])
    axes.plot(entry["Radius"], entry["Vcirc"], c=color[i], label=lab)
    axes.axvline(entry["Rmax"], c=color[i], linestyle='dashed')
    
plt.legend()
plt.tight_layout()

plt.savefig(os.path.join(path,filename), dpi=200)